In [1]:
#import dependencies

import keras
from keras.models import Sequential
from keras.utils import np_utils
from keras.preprocessing.image import ImageDataGenerator
from keras.layers import Dense, Activation, Flatten, Dropout, BatchNormalization
from keras.layers import Conv2D, MaxPooling2D
from keras.datasets import cifar10
from keras import regularizers, optimizers
import numpy as np
from matplotlib import pyplot

Using TensorFlow backend.


In [2]:
# download and split the data
(x_train, y_train), (x_test, y_test) = cifar10.load_data()
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')

print("training data = ", x_train.shape)
print("testing data = ", x_test.shape)

training data =  (50000, 32, 32, 3)
testing data =  (10000, 32, 32, 3)


In [3]:
# break training set into training and validation sets
(x_train, x_valid) = x_train[5000:], x_train[:5000]
(y_train, y_valid) = y_train[5000:], y_train[:5000]

print("training data = ", x_train.shape)
print("validation data = ", x_valid.shape)
print("testing data = ", x_test.shape)

training data =  (45000, 32, 32, 3)
validation data =  (5000, 32, 32, 3)
testing data =  (10000, 32, 32, 3)


In [4]:
# Normalize the data to speed up training
mean = np.mean(x_train)
std = np.std(x_train)
x_train = (x_train-mean)/(std+1e-7)
x_valid = (x_valid-mean)/(std+1e-7)
x_test = (x_test-mean)/(std+1e-7)

In [5]:
# one-hot encode the labels in train and test datasets
# we use “to_categorical” function in keras 

num_classes = 10
y_train = np_utils.to_categorical(y_train,num_classes)
y_valid = np_utils.to_categorical(y_valid,num_classes)
y_test = np_utils.to_categorical(y_test,num_classes)


In [6]:
# let's display one of the one-hot encoded labels
y_valid[0]

array([0., 0., 0., 0., 0., 0., 1., 0., 0., 0.], dtype=float32)

### data augmentation

In [10]:
# data augmentation
datagen = ImageDataGenerator(
    featurewise_center=False,
    samplewise_center=False,
    featurewise_std_normalization=False,
    samplewise_std_normalization=False,
    zca_whitening=False,
    rotation_range=15,
    width_shift_range=0.1,
    height_shift_range=0.1,
    horizontal_flip=True,
    vertical_flip=False
    )

# compute the data augmentation on the training set
datagen.fit(x_train)

In [11]:
x_train.shape

(45000, 32, 32, 3)

## download VGG model

In [7]:
# build the VGG16 network
from keras import applications

vgg_16 = applications.VGG16(include_top=False, weights='imagenet')

In [8]:
vgg_16.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, None, None, 3)     0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, None, None, 64)    1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, None, None, 64)    36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, None, None, 64)    0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, None, None, 128)   73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, None, None, 128)   147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, None, None, 128)   0         
__________

In [9]:
bottleneck_features_train = vgg_16.predict(x_train)

In [10]:
bottleneck_features_valid = vgg_16.predict(x_valid)

In [11]:
# save the bottleneck features as a Numpy array

np.save('bottleneck_features_train.npy', bottleneck_features_train) 
np.save('bottleneck_features_validation.npy', bottleneck_features_valid) 

In [12]:
datagen = ImageDataGenerator(
    featurewise_center=False,
    samplewise_center=False,
    featurewise_std_normalization=False,
    samplewise_std_normalization=False,
    zca_whitening=False,
    rotation_range=15,
    width_shift_range=0.1,
    height_shift_range=0.1,
    horizontal_flip=True,
    vertical_flip=False
    )
   
nb_train_samples = 45000  
num_classes = 10

# load training and validation bottleneck features
train_bottlenecks = np.load('bottleneck_features_train.npy')  
validation_bottlenecks = np.load('bottleneck_features_validation.npy')  


In [13]:
# create our FC model

model = Sequential()
model.add(Flatten(input_shape=(train_bottlenecks.shape[1:])))
model.add(Dense(256, activation='relu'))

model.add(Dropout(0.5))
model.add(Dense(10, activation='softmax'))

model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
flatten_1 (Flatten)          (None, 512)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 256)               131328    
_________________________________________________________________
dropout_1 (Dropout)          (None, 256)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 10)                2570      
Total params: 133,898
Trainable params: 133,898
Non-trainable params: 0
_________________________________________________________________


In [33]:
train_bottlenecks.shape[1:]

(1, 1, 512)

In [15]:
from keras.callbacks import ModelCheckpoint   

checkpointer = ModelCheckpoint(filepath='resnet.from.bottleneck.hdf5', save_best_only=True)

optimizer = keras.optimizers.rmsprop(lr=0.0003,decay=1e-6)
model.compile(loss='categorical_crossentropy', optimizer=optimizer, metrics=['accuracy'])

history = model.fit(train_bottlenecks, y_train, epochs=20,
          validation_data=(validation_bottlenecks, y_valid),
          callbacks=[checkpointer], verbose=2, shuffle=True)

Train on 45000 samples, validate on 5000 samples
Epoch 1/20
 - 2s - loss: 1.2924 - acc: 0.5559 - val_loss: 1.0459 - val_acc: 0.6324
Epoch 2/20
 - 2s - loss: 1.1283 - acc: 0.6200 - val_loss: 1.0411 - val_acc: 0.6554
Epoch 3/20
 - 2s - loss: 1.1010 - acc: 0.6347 - val_loss: 1.0352 - val_acc: 0.6552
Epoch 4/20
 - 2s - loss: 1.0972 - acc: 0.6436 - val_loss: 1.0568 - val_acc: 0.6672
Epoch 5/20
 - 2s - loss: 1.0928 - acc: 0.6512 - val_loss: 1.0719 - val_acc: 0.6690
Epoch 6/20
 - 2s - loss: 1.0956 - acc: 0.6572 - val_loss: 1.0881 - val_acc: 0.6686
Epoch 7/20
 - 2s - loss: 1.1008 - acc: 0.6585 - val_loss: 1.1030 - val_acc: 0.6646
Epoch 8/20
 - 2s - loss: 1.1074 - acc: 0.6611 - val_loss: 1.1342 - val_acc: 0.6654
Epoch 9/20
 - 2s - loss: 1.1135 - acc: 0.6634 - val_loss: 1.1526 - val_acc: 0.6668
Epoch 10/20
 - 2s - loss: 1.1077 - acc: 0.6676 - val_loss: 1.1694 - val_acc: 0.6710
Epoch 11/20
 - 2s - loss: 1.1168 - acc: 0.6691 - val_loss: 1.2178 - val_acc: 0.6688
Epoch 12/20
 - 2s - loss: 1.1185 - a

In [16]:
# training
batch_size = 64
epochs=20

from keras.callbacks import ModelCheckpoint   

checkpointer = ModelCheckpoint(filepath='.vgg16.transferlearning.20epochs.hdf5', verbose=1, save_best_only=True)

# you can try any of these optimizers by uncommenting the line
# optimizer = keras.optimizers.rmsprop(lr=0.001,decay=1e-6)
# optimizer = keras.optimizers.adam(lr=0.0005,decay=1e-6)

optimizer = keras.optimizers.rmsprop(lr=0.0003,decay=1e-6)
model.compile(loss='categorical_crossentropy', optimizer=optimizer, metrics=['accuracy'])
history = model.fit_generator(datagen.flow(train_bottlenecks, y_train, batch_size=batch_size), callbacks=[checkpointer],
                steps_per_epoch=train_bottlenecks.shape[0] // batch_size, epochs=epochs,verbose=2,
                validation_data=(validation_bottlenecks, y_valid))

/anaconda3/envs/tfdeeplearning/lib/python3.5/site-packages/keras_preprocessing/image.py:1643: UserWarning: NumpyArrayIterator is set to use the data format convention "channels_last" (channels on axis 3), i.e. expected either 1, 3, or 4 channels on axis 3. However, it was passed an array with shape (45000, 1, 1, 512) (512 channels).
  str(self.x.shape[channels_axis]) + ' channels).')


Epoch 1/20


KeyboardInterrupt: 